# __Fundamentos de Aprendizaje Automático 2021/2022__
## Práctica de Introducción: _TRATAMIENTO DE DATOS, PARTICIONAMIENTO Y DISEÑO PRELIMINAR DE LA APLICACIÓN_
### Grupo 1462 - Pareja 10 - Kevin de la Coba Malam

In [1]:
import pandas as pd
import numpy as np
from Datos import Datos
import EstrategiaParticionado

## __1. Introducción__
En este jupyter notebook se explicarán las soluciones a los diferentes apartados de la práctica de introducción. Para construir la aplicación se ha usado la librería pandas y numpy.

## __2. Clase Datos - Tratamiento de datos__
En el enunciado de la práctica se nos pide crear una clase "Datos" la cuál contenga los siguientes atributos:
* **nominalAtributos**: Lista de valores booleanos indicando si una columna es nominal o no.
* **diccionario**: Diccionario que nos permite _traducir_ los datos originales a datos númericos.
* **datos**: Matriz en la cual se guardan los datos _traducidos_.

Para resolver este apartado se ha creado el archivo __Datos.py__ en base a la plantilla proporcionada donde se encuentra la clase __Datos__. 
Dicha clase contiene un constructor que se encarga de construir los atributos antes mencionados, para eso lo primero que se hace es cargar el archivo _"tic-tac-toe.data"_ o _"german.data"_ usando la librería _pandas_, en concreto, usando la función _read_csv_:

In [2]:
df1 = pd.read_csv("ConjuntosDatos/tic-tac-toe.data")
df2 = pd.read_csv("ConjuntosDatos/german.data")

Como podemos ver, el archivo no es un _.csv_ pero el contenido que hay dentro _contiene el formato_ del _.csv_. Por esta razón la función no da ningún problema y se cargan los datos en el dataframe __df__. 

In [3]:
df1.head(5)

,TLeftSq,TMidSq,TRightSq,MLeftSq,MMidSq,MRightSq,BLeftSq,BMidSq,BRightSq,Class
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [4]:
df2.head(5)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A12,A13,A14,A15,A16,A17,A18,A19,A20,Class
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


Podemos ver que el dataframe ha cargado los archivos de forma exitosa. Podemos empezar a crear los atributos.

In [5]:
# Cargamos los dataset con los atributos
dataset1 = Datos("ConjuntosDatos/tic-tac-toe.data")
dataset2 = Datos("ConjuntosDatos/german.data")

## 2.1 nominalAtributos
Para cargar esta lista, se creo un método llamado __asignaNominalAtributos__. Dicho método recibe como argumento los tipos de las columnas (obtenidos usando __df.dtypes__). 
```py
def asignaNominalAtributos(self, tipos):
        """Itera sobre los tipos de atributos y asigna:
            True: Si la columna es nominal (object).
            False: Si la columna no es nominal.

        Args:
            line: Tipos obtenidos del dataframe de pandas.
        """
        for tipo in tipos:
            self.nominalAtributos.append(True if tipo == object else False)
```
Lo que se hace es que se itera sobre dichos elementos con el fin de ver si la columna contiene atributos nominales.


In [6]:
print("Dataset tic-tac-toe")
dataset1.nominalAtributos

Dataset tic-tac-toe


[True, True, True, True, True, True, True, True, True, True]

In [7]:
print("Dataset german")
dataset2.nominalAtributos

Dataset german


[True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False]

Podemos ver que el formato de las listas es el correcto. Podemos ver también que en el primer caso (tic-tac-toe) todos los atributos son nominales, pero en el segundo (german) __no__. La segunda columna, la quinta, etc. tienen valores numéricos y se muestran correctamente en la lista.

## 2.2 diccionario
Para cargar este diccionario se creo un método llamado __construyeDiccionario__. Dicho método recibe como argumento el dataframe original. 
```py
    def construyeDiccionario(self, df):
        """Método que itera sobre los datos y construye
        un diccionario de diccionarios en el que se muestra
        el valor que puede tener cada atributo. Ejemplo:
        {
            "Attr1": {"x": 1, "y": 2} -- Orden alfabético
            "Attr2": {"x": 1, "y": 2}
            ...
        }
        
        Args:
            df (Pandas Dataframe): Dataframe pandas con todos los datos.
        """
        for item in df.iteritems():
            columnName = item[0]
            possibleValues = list(df[columnName].unique()).copy()
            possibleValues.sort()
            for i, value in enumerate(possibleValues):
                if columnName not in self.diccionario:
                    self.diccionario[columnName] = {}
                self.diccionario[columnName][value] = i
```
El algoritmo implementado en el método itera sobre cada columna (con los datos de esta incluidos) mediante el métdo __df.iteritems()__. Una vez tenemos la columna lo que se hace es que se crea una lista con los todos los valores únicos de dicha columna mediante el método __df[column_name].unique()__. Esta lista se ordena y por último se añade un diccionario al diccionario "padre", donde la _key_ es el nombre de la columna y los _values_ son los distintos posibles valores ordenados alfabéticamente y enumerados. 

In [8]:
print("Dataset tic-tac-toe")
dataset1.diccionario['TLeftSq']

Dataset tic-tac-toe


{'b': 0, 'o': 1, 'x': 2}

In [9]:
print("Dataset german")
dataset2.diccionario['A1']

Dataset german


{'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}

Como se muestra estos son los posibles valores de la primera columna de cada uno de los dataset.

## 2.3 datos
Para cargar esta matriz se creo un método llamado __construyeDatos__. Dicho método recibe como argumento el dataframe original.
```py
    def construyeDatos(self, df):
        """Método que itera sobre todas las filas del dataset
        con el fin de traducir los datos a números.

        Args:
            df (Pandas Dataframe): Dataset con los datos originales.
        """
        self.datos = np.zeros(shape=df.shape)
        for i, item in enumerate(df.iterrows()):
            for j, col in enumerate(item[1].items()):
                self.datos[i][j] = self.diccionario[col[0]][col[1]]
```
El algoritmo implementado primero crea una matriz con la misma forma que tiene el dataframe recibido mediante la librería __numpy__, para inicializar dicha matriz se usa __np.zeros(shape=...)__ (crea una matriz con ceros) y se le envía como argumento la forma de la matriz con __df.shape__ (nos devuelve la forma del dataframe).

Una vez tenemos la matriz, iteramos sobre todas las filas mediante __df.iterrows()__, leemos cada columna en la fila y cambiamos el valor en la matriz numpy.

In [10]:
print("Dataset tic-tac-toe")
dataset1.datos

Dataset tic-tac-toe


array([[2., 2., 2., ..., 1., 1., 1.],
       [2., 2., 2., ..., 2., 1., 1.],
       [2., 2., 2., ..., 1., 2., 1.],
       ...,
       [1., 2., 1., ..., 1., 2., 0.],
       [1., 2., 1., ..., 1., 2., 0.],
       [1., 1., 2., ..., 2., 2., 0.]])

In [11]:
print("Dataset german")
dataset2.datos

Dataset german


array([[ 0.,  2.,  4., ...,  1.,  0.,  0.],
       [ 1., 29.,  2., ...,  0.,  0.,  1.],
       [ 3.,  8.,  4., ...,  0.,  0.,  0.],
       ...,
       [ 3.,  8.,  2., ...,  0.,  0.,  0.],
       [ 0., 27.,  2., ...,  1.,  0.,  1.],
       [ 1., 27.,  4., ...,  0.,  0.,  0.]])

Podemos ver que los datos se han traducido y unicamente tenemos valores numéricos.

## 2.4 Método extrae datos
Para construir el método extrae datos lo único que hacemos es crear una matriz numpy la cual contiene los índices que se nos especifican.
```py
def extraeDatos(self,idx):
    """Devuelve el subconjunto de los datos cuyos �ndices se pasan como argumento

    Args:
        idx (list): Lista que contiene los indices a recibir de los datos.

    Returns:
        Matriz numpy: Matriz numpy con los índices que se especifican.
    """
    datos = np.zeros(shape=(len(idx), self.datos.shape[1]))

    for i, index in enumerate(idx):
        datos[i] = self.datos[index]

    return datos
```

## __3. Particionamiento - Tratamiento de datos__
Se nos pide completar la plantilla donde tenemos una clase abstracta que define una _estrategia de particionado (EstrategiaParticionado)_. A parte se nos pide crear 2 clases más las cuales implementan estrategias de partcionado específicas, _Cross Validation (Validación cruzada)_ y _Simple Validation (Validación Simple)_.

La clase abstracta _EstrategiaParticionado_ fue completada unicamente declarando los atributos en el constructor

```py
class EstrategiaParticionado:
    """Clase abstracta donde se define la estrategia de particionado.
    """
  
    # Clase abstracta
    __metaclass__ = ABCMeta
  
    def __init__(self):
        """Constructor, solo se declaran los atributos.
        """
        self.particiones = []
    
    @abstractmethod
    def creaParticiones(self,datos,seed=None):
        """Método abstracto para crear particiones.

        Args:
            datos: Dataset.
            seed: Seed para generar aleatoriedad. Por defecto None.
        """
        pass
```
Como se puede ver la clase consta del atributo __particiones__, siguiendo el esquema proporcionado en el enunciado. Ese atributo contendrá objetos de la clase Particiones indicando los índices usados para el _Train_ y el _Test_. 

## 3.1 Validación simple
La validación simple parte los datos en 2, una parte para el test y otra para el training. La clase _ValidacionSimple_ recibe como argumentos en el constructor el __porcentaje usado para el test__ (el porcentaje es un número _float_ que esta entre (0-100)) y el número de ejecuciones que se harán (esto marca el número de permutaciones a crear para las particiones).

```py
class ValidacionSimple(EstrategiaParticionado):
    """Clase que define una estrategia de particionado,
    en concreto, validación simple.
    """
  
    def __init__(self, proporcionTest, numeroEjecuciones):
        """Constructor.

        Args:
            proporcionTest: Porcentaje para la proporción de test de los datos.
            numeroEjecuciones: Número de ejecuciones.
        """
        super().__init__()
        self.proporcionTest = proporcionTest
        self.numeroEjecuciones = numeroEjecuciones
```
Posteriormente el método _creaParticiones_ es implementado.
```py
def creaParticiones(self,datos,seed=None):
        def creaParticiones(self,datos,seed=None):
        random.seed(seed)
        self.particiones = []
        longitudDatos = np.shape(datos)[0]
        longitudTest = int((self.proporcionTest/100)*longitudDatos)
                
        lista_valores = [i for i in range(longitudDatos)]

        for i in range(self.numeroEjecuciones):
            self.particiones.append(Particion())
            
            # Calculamos los indices
            random.shuffle(lista_valores)
            
            # Asignamos los indices
            self.particiones[-1].indicesTest = lista_valores[:longitudTest]
            self.particiones[-1].indicesTrain = lista_valores[longitudTest:]
```
El algoritmo primero calcula la longitud de los datos (número de filas) y la longitud del test (número de filas asignadas al test). Debe tenerse en cuenta que para la longitud del test se hace una división entera. Posteriormente, creamos una lista de valores que contiene todos los índices de los datos, después, en un bucle, permutamos dicha lista y asignamos la parte correspondiente al test y la otra parte al training. Como la lista es permutada en cada iteración podemos coger siempre la primera parte como test y la segunda como training

Para probar el correcto funcionamiento crearemos un array numpy pequeño y cargaremos dichos índices.

In [12]:
datos = np.random.random((10, 2))

# Asignamos el 30% al test
vc = EstrategiaParticionado.ValidacionSimple(30, 3)
vc.creaParticiones(datos)

for i in range(3):
    print("\n\n")
    print("***Test***:", vc.particiones[i].indicesTest, "\n")
    print("***Train***:", vc.particiones[i].indicesTrain, "\n")




***Test***: [9, 8, 5] 

***Train***: [6, 2, 4, 0, 1, 7, 3] 




***Test***: [4, 3, 5] 

***Train***: [6, 0, 7, 2, 8, 9, 1] 




***Test***: [1, 5, 4] 

***Train***: [9, 6, 2, 7, 8, 3, 0] 



Se puede ver que los índices se crean correctamente.

Ahora se ejecutará lo mismo con el dataset de _tic-tac-toe_.

In [13]:
vc = EstrategiaParticionado.ValidacionSimple(10, 5) 
vc.creaParticiones(dataset1.datos)

for i in range(5):
    print("\n\n")
    print("***Test***:", vc.particiones[i].indicesTest, "\n")
    print("***Train***:", vc.particiones[i].indicesTrain, "\n")




***Test***: [85, 204, 441, 369, 724, 260, 487, 192, 513, 828, 857, 396, 61, 42, 757, 553, 271, 790, 518, 131, 933, 633, 552, 234, 219, 490, 632, 45, 180, 556, 675, 613, 23, 913, 170, 197, 551, 861, 448, 335, 68, 701, 289, 307, 265, 720, 872, 282, 255, 115, 923, 80, 94, 937, 291, 673, 238, 174, 126, 882, 110, 904, 427, 902, 376, 694, 235, 639, 224, 705, 782, 0, 169, 49, 509, 54, 493, 554, 136, 101, 14, 371, 810, 223, 787, 129, 364, 931, 435, 695, 524, 851, 577, 168, 525] 

***Train***: [46, 938, 863, 320, 758, 433, 663, 470, 372, 816, 845, 779, 785, 592, 885, 422, 749, 389, 243, 764, 451, 821, 645, 331, 537, 952, 820, 849, 173, 853, 346, 941, 261, 150, 315, 121, 50, 287, 383, 105, 65, 920, 26, 142, 432, 652, 207, 145, 791, 880, 928, 464, 731, 588, 847, 932, 839, 618, 583, 330, 79, 710, 566, 327, 325, 799, 399, 274, 326, 505, 542, 82, 102, 114, 914, 586, 687, 33, 124, 686, 555, 158, 473, 4, 430, 257, 305, 612, 515, 793, 57, 760, 77, 597, 495, 288, 693, 563, 624, 703, 729, 889, 116, 84

## 3.1 Validación cruzada
En esta estrategia dividimos los datos en un número N de partes, la intención usar cada una de las N partes como test y a la vez usar el resto como training.
La clase _ValidacionCruzada_ recibe como argumento en el constructor, el número de porciones que van ha haber en los datos.

```py
class ValidacionCruzada(EstrategiaParticionado):
    """Clase que define una estrategia de particionado,
    en concreto, validación simple.
    """

    def __init__(self, numeroParticiones):
        """Construcor.

        Args:
            numeroParticiones (int): Número de particiones de la validación cruzada.
        """
        super().__init__()
        self.numeroParticiones = numeroParticiones
```
Como se puede ver, simplemente en el constructor asignamos el valor de dicho atributo.
Posteriormente se implementó el método _creaParticiones_.

```py
def creaParticiones(self,datos,seed=None):
    random.seed(seed)
    self.particiones = []
    longitudDatos = np.shape(datos)[0]
    longitudPorcion = int(longitudDatos/self.numeroParticiones)

    lista_valores = [i for i in range(longitudDatos)]
    random.shuffle(lista_valores)

    for i in range(self.numeroParticiones):
        self.particiones.append(Particion())

        # Calculamos los indices del test
        fromTest = i*longitudPorcion
        toTest = fromTest + longitudPorcion

        # Asignamos los indices
        self.particiones[-1].indicesTest = lista_valores[fromTest:toTest]
        self.particiones[-1].indicesTrain = [i for i in lista_valores if i not in self.particiones[-1].indicesTest]
```
Al igual que en el método de validación simple, creamos una lista con los valores de los indices en los datos, posteriormente permutamos dicha lista pero en este caso solo es necesario hacerlo una vez. En un bucle asignamos las particiones usando la lista permutada. Hay que tener en cuenta que la asignación de las porciones se hace moviendonos sobre la lista permutada. Por ejemplo, si tuviesemos 4 porciones de los datos los cuales tienen 10 filas, se crearía una lista con los números enteros del 0-9, después permutaríamos la lista y por último en cada iteración del bucle iríamos asignando 2 porciones a cada test.
Iteración 1: Test - 2 primeros elementos de la lista permutada - Train - Resto.
Iteración 2: Test - 3er y 4º elementos de la lista permutada - Train - Resto.
...
Se ejecutará un ejemplo aquí debajo.

In [14]:
datos = np.random.random((8, 2)) 

vc = EstrategiaParticionado.ValidacionCruzada(4)
vc.creaParticiones(datos)
      
for i in range(4):
    print("\n\n")
    print("***Test***:", vc.particiones[i].indicesTest, "\n")
    print("***Train***:", vc.particiones[i].indicesTrain, "\n")




***Test***: [3, 6] 

***Train***: [2, 0, 5, 4, 1, 7] 




***Test***: [2, 0] 

***Train***: [3, 6, 5, 4, 1, 7] 




***Test***: [5, 4] 

***Train***: [3, 6, 2, 0, 1, 7] 




***Test***: [1, 7] 

***Train***: [3, 6, 2, 0, 5, 4] 



Se puede apreciar que el array es el mismo, solo vamos asignando la porcion del test que le toca.

Ahora se ejecutará lo mismo con el dataset de _tic-tac-toe_.

In [15]:
vc = EstrategiaParticionado.ValidacionCruzada(10) 
vc.creaParticiones(dataset1.datos)

for i in range(10):
    print("\n\n")
    print("***Test***:", vc.particiones[i].indicesTest, "\n")
    print("***Train***:", vc.particiones[i].indicesTrain, "\n")




***Test***: [779, 86, 88, 594, 24, 99, 926, 803, 603, 13, 50, 890, 322, 358, 510, 102, 853, 613, 839, 662, 456, 74, 414, 590, 63, 225, 334, 70, 491, 632, 723, 138, 395, 894, 267, 748, 941, 189, 296, 111, 399, 533, 368, 600, 626, 588, 167, 12, 509, 496, 592, 913, 376, 404, 93, 229, 350, 29, 271, 469, 570, 931, 476, 332, 610, 443, 153, 175, 583, 500, 77, 825, 756, 381, 652, 781, 842, 772, 460, 911, 226, 331, 431, 511, 651, 360, 916, 338, 354, 702, 593, 128, 172, 923, 614] 

***Train***: [83, 305, 211, 677, 951, 217, 299, 105, 401, 648, 612, 188, 672, 377, 520, 801, 241, 525, 378, 675, 255, 230, 186, 256, 902, 101, 231, 813, 220, 398, 841, 306, 423, 607, 342, 503, 548, 523, 553, 119, 905, 471, 766, 200, 867, 690, 765, 300, 683, 87, 222, 184, 283, 513, 297, 826, 809, 582, 901, 264, 488, 640, 208, 278, 817, 575, 686, 499, 243, 660, 692, 597, 877, 852, 783, 486, 42, 213, 664, 402, 82, 157, 78, 237, 339, 752, 348, 789, 650, 676, 865, 137, 531, 857, 918, 873, 847, 304, 183, 219, 430, 127, 3

## 4. Diseño preliminar de la aplicación
Se nos aconsejó implementar parte de la aplicación para entrenar modelos y comprobar el error producido en estos, como en esta práctica no se nos pide crear ningún modelo, unicamente se ha implementado el método __error__ del archivo clasificador, el método es muy simple, lo único que hace es comprobar si la predicción recibida es correcta comparando cada fila de la predicción con el elemento correspondiente en los datos (normalmente la clase). 
```py
def error(self, datos, pred):
    """Obtiene el numero de aciertos y errores para calcular la tasa de fallo

    Args:
        datos: Matriz numpy con los datos de entrenamiento
        pred: Predicción
    """
    errores = 0

    for i in range(datos.datos.shape[0]):
        if datos[i][-1] != pred[i]:
            errores += 1

    return (errores/datos.datos.shape[0])*100
```
La razón por la que no se ha implementado el método __validacion__ es porque al no tener ningún clasificador implementado es complicado construir el método sin que en el futuro de errores (como los podría dar la implementación del método error). Es por eso que la implementación de ambos métodos se hará en la siguiente práctica.